In [1]:
import copy
import math
import random
import warnings
from datasets import Dataset, DatasetDict, load_dataset
import evaluate
import json
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn.utils.class_weight import compute_class_weight
from torch import nn
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm
from transformers import (AutoModelForMaskedLM, AutoModelForQuestionAnswering, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments)

In [2]:
# Wczytuję dane poquad
with open("poquad-train.json", 'r') as file:
    train_data = json.load(file)['data']
with open("poquad-dev.json", 'r') as file:
    val_data = json.load(file)['data']

In [3]:
# Tworzę dataset z danych poquad
def proccess_the_data(data):
    input_data = []
    [
        [
            [
                input_data.append(
                    [paragraph_element["context"], question["question"], question["answers"][0]]
                ) if "answers" in question.keys() else input_data.append(
                    # czasami w danych nie ma pola"answers", jest "plausible_answers", wtedy wczytuję plasuible_answers
                    [paragraph_element["context"], question["question"], question["plausible_answers"][0]]
                ) 
                for question in paragraph_element["qas"]
            ] 
            for paragraph_element in element["paragraphs"]
        ] 
        for element in data
    ]
    
    return [[str(id), element[0], element[1], element[2]["answer_start"], element[2]["answer_end"], element[2]["text"]] for id, element in enumerate(input_data)]

# Tworzę dataset testowy w wersji raw, czyli niestokenizowanej
train_dataset_dataframe = pd.DataFrame(proccess_the_data(train_data), columns=["id", "context", "question", "answer_start", "answer_end", "answer_text"])
train_dataset_raw = Dataset.from_pandas(train_dataset_dataframe)

# Tworzę dataset validacyjny w wersji raw, czyli niestokenizowanej
val_dataset_dataframe = pd.DataFrame(proccess_the_data(val_data), columns=["id", "context", "question", "answer_start", "answer_end", "answer_text"])
val_dataset_raw = Dataset.from_pandas(val_dataset_dataframe)

In [4]:
# Funkcja dostosowująca dataset do postaci, która może zostać użyta do trenowania modelu
def preprocess_data(data, tokenizer, stride, max_length):
    # dane zostają stokenizowane
    tokenized_data = tokenizer(
        data["question"],
        data["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    offset_mapping = tokenized_data["offset_mapping"]
    sample_map = tokenized_data.pop("overflow_to_sample_mapping")
    # model w trakcie uczenia będzie zwracał początek i koniec fragmentu kontekstu, który odpowiada na zadane pytanie
    # Poniżej przygotowuję listy, w których dla każdego stokeniozowanego przykładu będę zapisywał od którego tokenu się zaczyna
    # i na którym tokenie kończy się odpowiedź na zadane pytanie
    
    # Dodatkową trudnością jest sytuacjia, gdy kontekst i pytanie mają długość większą niż 300 tokenów, zostaną wtedy podzielone na 2 przypadki uczące,
    # może przez to dojść do sytuacji, gdy we fragmencie kontekstu nie znajdzie się odpowiedź na zadane pytanie. W takiej sytuacji jako zarówno początek
    # jak i koniec odpowiedzi zostanie ustawiona wartość 0.
    start_positions = []
    end_positions = []

    # iteruję po przykładach uczących wyznaczając początek i koniec odpowiedzi.
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        start_char = data["answer_start"][sample_idx]
        end_char = data["answer_end"][sample_idx]
        sequence_ids = tokenized_data.sequence_ids(i)

        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    tokenized_data["start_positions"] = start_positions
    tokenized_data["end_positions"] = end_positions
    return tokenized_data

In [5]:
# Przekształcam dartaset treningowy na huggingface'owy dataset, który nadaje się do wykorzystania w treningu
stride = 70
max_length = 300
model_checkpoint = "allegro/plt5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

preproccess_lambda = lambda x: preprocess_data(x, tokenizer, stride, max_length)

train_dataset = train_dataset_raw.map(
    preproccess_lambda,
    batched=True,
    remove_columns=train_dataset_raw.column_names,
)

len(train_dataset), len(train_dataset_raw)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/56618 [00:00<?, ? examples/s]

(65556, 56618)

In [6]:
# Przekształcam dartaset walidacyjny na huggingface'owy dataset, który nadaje się do wykorzystania w treningu
# oraz w testach modelu
stride = 70
max_length = 300

preproccess_lambda = lambda x: preprocess_data(x, tokenizer, stride, max_length)

val_dataset = val_dataset_raw.map(
    preproccess_lambda,
    batched=True,
    remove_columns=val_dataset_raw.column_names,
)

len(val_dataset), len(val_dataset_raw)

Map:   0%|          | 0/7060 [00:00<?, ? examples/s]

(8164, 7060)

In [7]:
# Poniżej hiperparametry, które zostały wykorzystane do trenowania modelu. Trening został przeprowadzony na 3 epokach, zgodnie
# z wymaganiami wyszczególnionymi w treści zadania.

torch.cuda.empty_cache()
#checkpoint = "allegro/plt5-base"
checkpoint = "finetuning-test-full/checkpoint-4097"
#model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)

args = TrainingArguments(
    "finetuning-test-full",
    evaluation_strategy="epoch",
    save_strategy="steps",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    warmup_steps=600,
    save_steps = 1000
)

In [8]:
# poniżej trainer, któy dokonał treningu

# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     tokenizer=tokenizer,
# )

# poniższy kod pozwolił mi na wznowienie treningu z checkpointa. Dzięki temu byłem w stanie wykonać cały trening 3 epok, który łącznie zajął na 
# moim sprzęcie ponad 12 godzin.
# trainer.train(resume_from_checkpoint=True)

In [9]:
# Poniżej kod, który na podstawie datasetu "simple legal questions" tworzy dataset, który może zostać wykorzystany do
# przetestowania wytrenowanego modelu. Cała operacja jest dość skomplikowna i wymaga czytania z 4 plików, a następnie
# połączenia danych.

# W poniższym pliku znajdują się id pytań, oraz odpowiedzi na nie.
with open("answers.jl", 'r') as file:
    data_answers_test_answers = [json.loads(line) for line in file]

data_answers_test_answers = pd.DataFrame(data_answers_test_answers)
data_answers_test_answers = data_answers_test_answers[data_answers_test_answers["score"] == "1"]

# W poniższym pliku znajdują się teksty, będące kontekstem
with open("passages.jl", 'r') as file:
    data_answers_test_passages = [json.loads(line) for line in file]
data_answers_test_passages = pd.DataFrame(data_answers_test_passages)

# W ponizszym pliku znajduje się ralacja pomiędzy id pytania, a kontekstem, do którego zostaje zadane pytanie
with open("relevant.jl", 'r') as file:
    data_answers_test_relevant = [json.loads(line) for line in file]
data_answers_test_relevant = pd.DataFrame(data_answers_test_relevant)

# W poniższym pliku znajdują treści pytań w relacji do ich id
with open("questions.jl", 'r') as file:
    data_answers_test_questions = [json.loads(line) for line in file]
data_answers_test_questions = pd.DataFrame(data_answers_test_questions)


# poniżej pola które będzie zawierał gotowy dataset
#[id, question, context, answer_text]

data_test_dataframe = []

# Iteruję po id pytania, na które znajduje się odpowiedź, a następnie 
for question_id in data_answers_test_answers["question-id"].unique():
    context_id = data_answers_test_relevant[data_answers_test_relevant["question-id"] == question_id]["passage-id"].values[0]
    context = data_answers_test_passages[data_answers_test_passages["_id"] == context_id]["text"].values[0]
    question = data_answers_test_questions[data_answers_test_questions["_id"] == question_id]["text"].values[0]
    answer_text = data_answers_test_answers[data_answers_test_answers["question-id"] == question_id]["answer"].values[0]
    data_test_dataframe.append([question_id, question, context, answer_text])

data_test_dataframe = pd.DataFrame(data_test_dataframe, columns=["id", "question","context", "answer_text"])
data_test_raw = Dataset.from_pandas(data_test_dataframe)

In [10]:
# Funkcja wczytująca model, do abstrakcji jaką jest trainer.
def get_trainer(source):
    model = AutoModelForQuestionAnswering.from_pretrained(source)
    # Training args nie mają znaczenia. Abstrakcja trainer, jest używana jeydnie jako wygodny pojemnik na model, do wywołania funkcji
    # trainer.predict(Datset).
    training_args = TrainingArguments(output_dir="./results", per_device_eval_batch_size=1)
    trainer = Trainer(model,training_args)
    return trainer

#source = "apohllo/plt5-base-poquad"
source = "Checkpoint-wtorek"

trainer2 = get_trainer(source)

In [11]:
# Funkcja dostosowująca dataset testowy, w taki sposób, aby był używalny przez model.
def preprocess_input(data, stride, max_length):
    tokenized_data = tokenizer(
        data["question"],
        data["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    return tokenized_data

stride = 70
max_length = 300

process_input_lambda = lambda x: preprocess_input(x, stride, max_length)

test_dataset = data_test_raw.map(
    process_input_lambda,
    batched=True,
    remove_columns=data_test_raw.column_names,
)

len(test_dataset), len(data_test_raw)

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

(542, 484)

In [12]:
# Uzyskuję wyniki dla walidacyjnego datasetu
results = trainer2.predict(test_dataset)
predictions, _, _ = results
start_logits, end_logits, _ = predictions

In [13]:
# Funkcja służąca do obliczania metryk exact match i f1 score.
def compute_metrics(start_logits, end_logits, tokenized_data, references, n_best, max_answer_length):
    predicted_answers = []
    tokenized_words = []
    ids = []
    
    # Iteruję po datasecie, dla każdego z przykładów zostanie stworzony słownik zawierający: id przykładu
    # oraz odpowiedź na pytanie, najbardziej prawdopodobną według modelu.
    for index, example in tqdm(enumerate(references)):
        example_id = example["id"]
        context = example["context"]
        answers = []
        
        start_logit = start_logits[index]
        end_logit = end_logits[index]
        offsets = tokenized_data[index]["offset_mapping"]
        tokenized_words.append([(context[start : end]) for start, end in list(offsets)])

        # uzyskuję n_best id początku odpowiedzi oraz n_best id końca odpowiedzi.
        # uzyskiwane są, poprzez wybranie 20 id, pod którymi znajdują się największe wartości przewidywane przez model.
        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        
        # Iteruję przez wszystkie wybrane początki odpowiedzi, oraz wszystkie końce odpowiedzi.
        # Wyciągam z kontekstu fragment od znaku start_index, aż do end_index i następnie dodaję do listy "answers"
        for start_index in start_indexes:
            for end_index in end_indexes:
                # W lisćie offsets znajdują się początki każdego z tokenów.
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # max_answer_length to podana jako parametr maksymalna liczba tokenów w odpowiedzi, aby była ona rozpatrywana
                if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                    continue
                if start_index == 0 and end_index == 0:
                    continue

                answer = {
                    "text": context[offsets[start_index][0] : offsets[end_index][1]],
                    "logit_score": start_logit[start_index] + end_logit[end_index],
                    "ids": [start_index, end_index]
                }
                answers.append(answer)
        # dla konkretnego elementu, wybierany jest najbardziej prawdopodobna predykcja
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
            ids.append(best_answer["ids"])
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": {"text": [ex["answer_text"]], "answer_start": [1]}} for ex in references]
    metric = evaluate.load("squad")
    return (predicted_answers, theoretical_answers, metric.compute(predictions=predicted_answers, references=theoretical_answers), tokenized_words)

In [14]:
res = compute_metrics(start_logits, end_logits, test_dataset, data_test_raw, 20, 60)
res[2]

0it [00:00, ?it/s]

{'exact_match': 0.0, 'f1': 10.683034201763867}

In [15]:
for i in range(0, 10):
    print("Poprnawna odpowiedź:", res[1][i]["answers"]["text"][0])
    print("Odpowiedź zwrócona przez model:", res[0][i]["prediction_text"])
    print("-----------------------------------------------------------------------------------------")
    print()

Poprnawna odpowiedź: Tak, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.
Odpowiedź zwrócona przez model: Art. 345. § 1. Żołnierz, który dopuszcza się czynnej napaści na przełożonego, podlega karze aresztu wojskowego albo pozbawienia wolności do lat 3.
-----------------------------------------------------------------------------------------

Poprnawna odpowiedź: Komisja przetargowa składa się z co najmniej trzech osób.
Odpowiedź zwrócona przez model: Art. 21. 1. Członków komisji przetargowej powołuje i odwołuje kierownik zamawiającego. 2. Komisja przetargowa
-----------------------------------------------------------------------------------------

Poprnawna odpowiedź: Komandytariusz odpowiada za zobowiązania spółki wobec jej wierzycieli tylko do wysokości sumy komandytowej.
Odpowiedź zwrócona przez model: Art. 111. Komandyt
-----------------------------------------------------------------------------------------

Poprnawna odpowiedź: Wartość rzeczowych składników m

Jak widać w przykładach wypisanych powyżej, zazwyczaj początek kontekstu stanowi zarazem początek odpowiedzi. Przeglądając odpowiedzi można zauważyć, że niektóre zawierają nieco sensu. Tak jest chociażby w odpowiedzi 1, która zawiera odpowiedź spójną ze wzorcową odpowiedzią. Warto tutaj również zaznaczyć, że dataset który został wykorzystany do odpowiedzi na pytania jest stworzony przez studentów i bardzo często odpowiedzi zostały przygotowane dla podejścia AQA, natomiast mój model został wytrenowany i jest przewidziany do korzystania z podejścia EQA, przez co wyniki mogą być gorsze niż są w rzeczywistości.

In [16]:
# Poniższy kod jest niezbędny do uzyskania wyników dla datasetu walidacyjnego. Wykonanie za pomocą
# abstrakcji jaką jest trainer, powoduje błąd związany z niewystarczającą ilością pamięci.
# Problem nie wstępuje, gdy wywołuję funkcję ewaluującą bezpośrednio na modelu.
val_dataset.set_format("torch")
eval_set_for_model = val_dataset.remove_columns(["offset_mapping"])


device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch = {k: eval_set_for_model[k].to(device) for k in eval_set_for_model.column_names}
trained_model = AutoModelForQuestionAnswering.from_pretrained("Checkpoint-wtorek").to(
    device
)

mini_batches = []
for i in tqdm(range(len(batch['input_ids']))):
    # Create a mini-batch for each element
    mini_batch = {k: v[i].unsqueeze(0).to(device) for k, v in batch.items()}
    mini_batches.append(mini_batch)

start_logits_arr = []
end_logits_arr = []

with torch.no_grad():
    for element_id in tqdm(range(len(mini_batches))):
        element = mini_batches[element_id]
        outputs = trained_model(**element)
        start_logits = outputs.start_logits.cpu().numpy()
        end_logits = outputs.end_logits.cpu().numpy()
        start_logits_arr.append(start_logits[0])
        end_logits_arr.append(end_logits[0])

  0%|          | 0/8164 [00:00<?, ?it/s]

  0%|          | 0/8164 [00:00<?, ?it/s]

In [17]:
res2 = compute_metrics(start_logits_arr, end_logits_arr, val_dataset, val_dataset_raw, 20, 60)

0it [00:00, ?it/s]

In [20]:
print(res2[2])

{'exact_match': 0.014164305949008499, 'f1': 8.3200053715511}


In [18]:
for i in range(0, 10):
    print("Poprnawna odpowiedź:", res2[1][i]["answers"]["text"][0])
    print("Odpowiedź zwrócona przez model:", res2[0][i]["prediction_text"])
    print("-----------------------------------------------------------------------------------------")
    print()

Poprnawna odpowiedź: kompilację poglądów różnych rabinów na określony temat
Odpowiedź zwrócona przez model: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu Mojżesz otrzymał od Boga całą Torę
-----------------------------------------------------------------------------------------

Poprnawna odpowiedź: dwóch
Odpowiedź zwrócona przez model: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu
-----------------------------------------------------------------------------------------

Poprnawna odpowiedź: pisanej, a drugą część w formie ustnej
Odpowiedź zwrócona przez model: Pisma rabiniczne – w tym Miszna – stanowią kompilację poglądów różnych rabinów na określony temat. Zgodnie z wierzeniami judaizmu
-----------------------------------------------------------------------------------------

Poprnawna odpowiedź: ustna
Odpowiedź zwróco

Dla datasetu walidacyjnego obserwacje są bardzo podobne, jak dla datasetu testowego. Tutaj warto zauważyć, że np. dla przykładów 1-4 odpowiedzi na pytania są dokładnie takie same, pomimo tego, że treść pytania jest inna. Warto zwrócić uwagę na poprawną odpowiedź dla pytania 5, gdzie poprawna odpowiedź to: **280 strzelców, kilkuset chłopów kosynierów i 60 kawalerzystów**, natomiast model udzielił: **Sformowany przez nią oddział partyzancki liczył 280 strzelców, kilkuset chłopów kosynierów i 60 kawalerz** co jest bardzo podobne. Według mnie model poradził sobie lepiej na datasecie walidacyjnym niż na datasecie testowym. Jest to zrozumiałe, gdyż pytania i kontekst są podobne do danych testowych. Natomiast dane w zbiorze testowym znacząco się od nich różnią.

## Does the performance on the validation dataset reflects the performance on your test set?

Jak można zobaczyć po metrykach, ich wyniki są bardzo podobne dla obu datasetów. Warto jednak pamiętać, że dataset testowy jest dużo mniejszy niż dataset walidacyjny, więc bardzo ciężko porównać te oba wyniki.

Kolejnym problemem jest wspomniany wcześniej fakt, że dataset testowy został stworzony przez studentów i jest bardziej datasetem dla problemu AQA, niż wykorzystany w tym laboratorium EQA.

## What are the outcomes of the model on your test questions? Are they satisfying? If not, what might be the reason for that?

Wyniki nie są zadowalające. Niestety, pomimo starań nie dysponuję sprzętem na tyle silnym, aby być w stanie wytrenować model wystarczająco dobrze, aby był w stanie odpowiadać na zadane pytania w zadowalający sposób. Przy większych zasobach na pewno możliwe byłoby stworzenie modelu, który w zadowalający sposób odpowiedziałby na zadane pytania.

## Why extractive question answering is not well suited for inflectional languages?

Wydaje mi się, że największym problemem jest różnica w formie słów, które są wymagane przez sposób zadania pytania, w stosunku do tego w jakiej formie słowa występują w kontekście. Model może mieć przez to problem ze znalezieniem odpowiedniego fragmentu kontekstu, który odpowiada na zadane pytanie, a nawet jeżeli zostanie on poprawnie znaleziony, to korzystanie z takiej odpowiedzi może być niekomfortowe.